In [1]:
MNEMONIC = "ensure model kiss provide sense shadow umbrella oyster switch success milk unaware tree blush clutch bleak upgrade always below favorite fork discover pattern cinnamon"

from bip_utils import Bip39SeedGenerator, Bip84Coins, Bip84, Bip44Changes
import requests

def check_btc_balance(address: str):
    """Check Bitcoin balance for an address using Blockchain.com API."""
    try:
        response = requests.get(f"https://blockchain.info/balance?active={address}")
        data = response.json()
        return data[address]["final_balance"] / 100000000  # Convert from satoshis to BTC
    except Exception as e:
        print(f"Error checking balance: {e}")
        return 0

def generate_bip84_addresses(mnemonic: str, passphrase: str = "",
                            coin_type=Bip84Coins.BITCOIN, count: int = 10):
    """Generate BIP84 addresses and check their balances."""

    save = []
    # Generate seed
    seed_bytes = Bip39SeedGenerator(mnemonic).Generate(passphrase)

    # Create BIP84 wallet
    bip84_wallet = Bip84.FromSeed(seed_bytes, coin_type)

    # Get Bip84 object for receiving accounts (BIP84: account 0 for receiving)
    bip84_account = bip84_wallet.Purpose().Coin().Account(0)

    print(f"Master Public Key (xpub): {bip84_account.PublicKey().ToExtended()}")

    # Generate addresses and check balances
    for i in range(count):
        # Use Bip44Changes.CHAIN_EXT for receiving addresses (path m/84'/0'/0'/0/i)
        bip84_address = bip84_account.Change(Bip44Changes.CHAIN_EXT).AddressIndex(i)
        address = bip84_address.PublicKey().ToAddress()

        # Check balance (using Blockchain.com API as example)
        balance = check_btc_balance(address)
        save.append([address, balance, passphrase])
        # print(f"Address {i}: {address} | Balance: {balance} BTC")
        if balance != 0:
            print(f'Found passphrase:{passphrase}, balance: {balance}')
            break
    return save

In [2]:
import random
import itertools

def generate_typos(original):
    while True:
        # Choose a random error type
        error_type = random.choice([
            'swap_adjacent',
            'missing_char',
            'extra_char',
            'wrong_char',
            'case_swap',
            'repeat_char',
            'missing_symbol'
        ])

        # Convert to list for easier manipulation
        chars = list(original)

        # Apply the selected error
        try:
            if error_type == 'swap_adjacent' and len(chars) > 1:
                # Swap two adjacent letters
                pos = random.randint(0, len(chars)-2)
                chars[pos], chars[pos+1] = chars[pos+1], chars[pos]

            elif error_type == 'missing_char' and len(chars) > 1:
                # Remove a random character
                pos = random.randint(0, len(chars)-1)
                del chars[pos]

            elif error_type == 'extra_char':
                # Add a random character
                pos = random.randint(0, len(chars))
                extra_char = random.choice('abcdefghijklmnopqrstuvwxyz0123456789*')
                chars.insert(pos, extra_char)

            elif error_type == 'wrong_char':
                # Replace a character with a similar one
                pos = random.randint(0, len(chars)-1)
                if chars[pos].isdigit():
                    # For digits, replace with a nearby key
                    wrong = str((int(chars[pos]) + random.choice([-1, 1]))[-1])
                elif chars[pos].isalpha():
                    # For letters, replace with a nearby keyboard character
                    if chars[pos].lower() in 'qwertyuiopasdfghjklzxcvbnm':
                        keyboard_row1 = 'qwertyuiop'
                        keyboard_row2 = 'asdfghjkl'
                        keyboard_row3 = 'zxcvbnm'
                        if chars[pos].lower() in keyboard_row1:
                            idx = keyboard_row1.index(chars[pos].lower())
                            wrong = random.choice(keyboard_row1[max(0, idx-1):min(len(keyboard_row1), idx+2)])
                        elif chars[pos].lower() in keyboard_row2:
                            idx = keyboard_row2.index(chars[pos].lower())
                            wrong = random.choice(keyboard_row2[max(0, idx-1):min(len(keyboard_row2), idx+2)])
                        else:
                            idx = keyboard_row3.index(chars[pos].lower())
                            wrong = random.choice(keyboard_row3[max(0, idx-1):min(len(keyboard_row3), idx+2)])
                    else:
                        wrong = random.choice('abcdefghijklmnopqrstuvwxyz')
                else:
                    # For symbols, replace with another symbol
                    wrong = random.choice('*#@!$%&')

                # Maintain original case
                if chars[pos].isupper():
                    wrong = wrong.upper()
                chars[pos] = wrong

            elif error_type == 'case_swap':
                # Change the case of a random character
                pos = random.randint(0, len(chars)-1)
                if chars[pos].isalpha():
                    chars[pos] = chars[pos].upper() if chars[pos].islower() else chars[pos].lower()

            elif error_type == 'repeat_char':
                # Repeat a random character
                pos = random.randint(0, len(chars)-1)
                chars.insert(pos, chars[pos])

            elif error_type == 'missing_symbol':
                # Remove the symbol if it exists
                if '*' in chars:
                    chars.remove('*')

        except:
            # If any error occurs (like trying to remove from empty list), try again
            continue

        # Return the generated variation
        yield ''.join(chars)

# Example usage: generate 20 variations
original = "example"  # You should define 'original' before using it
typo_generator = generate_typos(original)

In [3]:
print("Variações com erros de digitação:")
passphrase = []
for i, typo in enumerate(itertools.islice(typo_generator, 100000000)):
    passphrase.append(typo)

Variações com erros de digitação:


In [4]:
save_final = []
for PASSPHRASE in passphrase:
    save = generate_bip84_addresses(MNEMONIC, PASSPHRASE, Bip84Coins.BITCOIN, 10)
    save_final = save_final + save


Master Public Key (xpub): zpub6qr6qStfBKHt74eemGcBdGpJy7qm6eXdNm7apw3nZwnrunB31mmCDZu5c2UMS3mswRher6GWV8LQSWtTjGu94ea3SW1R5hEWX6f2hFMPo4t
Master Public Key (xpub): zpub6s4DKZGw5yVvPuVRu95cPwTMz5UrqMSJXbagEeW17H6fSwAoiKJgQYf7ZVZwkQ87ihQv1ZJeZxuDWtUShtkWejMco9z856ZgbkqbvwC59cP
Master Public Key (xpub): zpub6qzG1o9SujU2RksZrXtAULvCwyDpfQqDdFBGrrmYpfqwtLwZstZZqS45VTc8Y4rJ4P7Ei9vnmKmRYmQ4HqRzcaJE6LY6qGZxJ2BG62VcDCp
Master Public Key (xpub): zpub6qygbc1FWXPujAwZonWpGmv5A1gionGPo6tSmiubFCFuu3Teq1dq7kXzK5VW3wkejVnNGjeoBSwGqnp4GyNAVRdgrsLYEDHiKaphMbeQSAE
Master Public Key (xpub): zpub6rcSStX1Hs8nuBvqWDfdXtWpAQpYoSxWSdZQKMSgFxhtSyAZLRSnZAG3y69cvpLnEm7KQbXaCtMG2MQwfAQ1KAFBNGTUBJpe8tVfEoM9Kx9
Master Public Key (xpub): zpub6qzG1o9SujU2RksZrXtAULvCwyDpfQqDdFBGrrmYpfqwtLwZstZZqS45VTc8Y4rJ4P7Ei9vnmKmRYmQ4HqRzcaJE6LY6qGZxJ2BG62VcDCp
Master Public Key (xpub): zpub6rk5k43kbEYhTMbMz8od14kKaxBgfsVStKEV5FWxThjojubuWu4BRciwu6NGpJJ6rcKYFet3koJ4CAyhisoQwMupzKuAgbf87NygYToRXRK
Master Public Key (xpub): zpub6r1t

KeyboardInterrupt: 